# Doorrekenen van een neuraal netwerk

In dit document ga ik een neuraal netwerk doorrekenen vanuit een JSON bestand.

## De achterliggende wiskunde

### Matrix vermenigvuldiging

Voor het doorekenen van het neurale netwerk maken wij gebruik van matrix vermenigvuldiging.

het vermenigvuldigen van een matrix werkt als volgt.

$
 A = \begin{bmatrix}
    1 & 2 \\
    3 & 4 \\
    5 & 6 \\
\end{bmatrix}
$

$
B = \begin{bmatrix}
    1 & 2 & 3 \\
    4 & 5 & 6 \\
\end{bmatrix}
$


Bij deze 2 matrices is de vermenigvuldiging A * B mogelijk. omdat het aantal kolommen van de eerste matrix gelijk is aan het aantal rijen van de tweede matrix.

Als dit niet zo is, is een vermenigvuldiging niet mogelijk.

De uitkomst van een vermenigvuldiging tussen 2 matrices is altijd een matrix met de hoogte van de eerste matrix en de breedte van de tweede matrix.

Voor het uitrekenen van een $X$-$Y$ positie in de nieuwe matrix moet je rij $X$ van $A$ vermnigvuldigen met kolom $Y$ van $B$.

Voor bijvoorveeld de positie $X = 2$ en $Y = 2$ in de nieuwe matrix is het.

$3 * 2 + 4 * 5 = 26$

Dit moet je doen voor alle posities.

Bron: https://www.wisfaq.nl/pagina.asp?nummer=1828

### Neuraal netwerk uitrekenen 

Het neurale netwerk bestaat uit meerdere layers, deze layers hebben weights. Een layer kan worden gezien als een matrix en de weights als een rij getallen in de matrix. 

Voor het uitrekenen van het neurale netwerk moeten wij voor iedere layer een multiplicatie doen met de vorige layer. Dit geeft ons het resultaat van de huidige layer en de input voor de volgende layer. Als wij dit doen tot de laatste layer krijgen wij de output van het neurale netwerk.

De volgorde waarop wij dit doen is van achter naar voren.
Wij beginnen met de "laatste layer" en lopen dan terug naar voren. De laatste verminigvuldiging is met de input vector.

Ook kan als optimalisatie de vermenigvuldigingen van de layers vooraf al worden gedaan. Deze kunnen dan worden opgeslagen en dan hoeft er alleen een vermenigvuldiging met de input vector te gebeuren. 

## De code

Wij beginnen met het includen van de JSON package, dit zorgt er voor dat wij een stuk makkelijker kunnen omgaan met JSON.

In [1]:
import json

### Configuratie

Hier kunnen de parameters van dit script worden ingesteld.

In [2]:
file_path = "examples/example-2layer.json"
input_vector = [1, 1, 1, 1, 1]

### De functies

De eerste en enige functie is de 'multiply' functie, deze functie vermenigvuldigd 2 matrices en geeft het resultaat van de vermenigvuldiging terug, dit is dan ook een belangerijk onderdeel van deze opdracht.

In het volgende stuk code word er gecontroleerd of de matrices de goede structuur hebben of dat ze nog moeten worden omhulst in een array.

Daarna gaan wij de Matrix maken die wij gaan returnen, deze matrix heeft de hoogte van de 'l_matrix' en de breedte van de 'r_matrix', ook kopieeren wij de 'result_row' omdat python er anders een refrence van maakt, als dat zou gebeuren zou iets aanpassen in de 1e rij er ook voor zorgen dat alle andere rijen worden aangepast naar hetzelfde getal.

Dan komt het belangerijkste stuk, de vermenigvuldiging.

En dan returnen wij de nieuwe matrix.

In [3]:
def multiply(l_matrix, r_matrix):
    result = []
    result_row = []
    
    # Make sure it has the correct structure, otherwise give it the correct structure
    if not hasattr(r_matrix[0], '__len__'):
        r_matrix = [r_matrix]

    # Make sure it has the correct structure, otherwise give it the correct structure
    if not hasattr(l_matrix[0], '__len__'):
        l_matrix = [l_matrix]
        
    for l in range(len(r_matrix[0])):
        result_row.append(0)

    for l in range(len(l_matrix)):
        result.append(result_row.copy())
        
    for l in range(len(l_matrix)):
        for r in range(len(r_matrix[0])):
            for rr in range(len(r_matrix)):
                result[l][r] += l_matrix[l][rr] * r_matrix[rr][r]
                
    return result

### Het script

Nu wij de functies hebben gehad kunnen wij naar het script.

Eerst openen wij het JSON bestand.

In [4]:
file = open(file_path, 'r')
json_data = json.load(file)

matrices = []

Hier gaan het JSON bestand parsen naar een formaat wat wij kunnen gebruiken.

De "[0] * int(output_layer_size)" zorgt ervoor dat wij een array krijgen van het juiste formaat, zodat wij deze hierna kunnen vullen.

In [5]:
for layer in json_data:
    matrix = []
    weights = json_data[layer]["weights"]
    output_layer_size = int(json_data[layer]["size_out"])

    for node_weights in weights:
        node = weights[node_weights]

        # Create a new vector with the length of the output size
        list_weights = [0] * int(output_layer_size)

        for weight in node:
            list_weights[int(weight) - 1] = float(node[weight])

        matrix.append(list_weights)
    matrices.append(matrix)

Nu hoeven wij alleen nog alle layers achter elkaar (van achter naar voren) te verminigvuldigen, dit kunnen wij doen met de eerder vermelde multiply functie.

De laatste stap is het vermenigvuldigen met de input_vector voor het resultaat.

In [6]:
result = None

if len(matrices) == 1:
    result = multiply(input_vector, matrices[0])
else:
    n_matrices = len(matrices)

    prev_result = multiply(matrices[-2], matrices[-1])

    for i in range(n_matrices - 2 , 0):
        prev_trans_matrix = multiply(matrices[i], prev_result)

    result = multiply(input_vector, prev_result)

En dan hebben wij het resultaat.

In [7]:
print(result)

[[0.62, 0.35999999999999993]]
